In [357]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(font_scale=2)

In [713]:
# Parameters

R0 = 3.3
AR = 0.8
C0 = 100
POP = 1e7
gamma = 1

In [714]:
# SIR model https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model

def ds(s,i,pop=POP,r0=R0,gamma=gamma):
    return -s*i*r0*gamma/pop

def di(s,i,pop=POP,r0=R0,gamma=gamma):
    return -ds(s,i,pop,r0,gamma) - gamma*i

def dr(i,gamma=gamma):
    return gamma*i

def simulate(tmax,i0,s0,pop=POP,r0=R0,gamma=gamma,step_size=1e-5):
    ins = [i0]
    ss = [s0]
    for t in range(int(tmax/step_size)):
        ins.append(ins[-1] + step_size*di(ss[-1],ins[-1],pop,r0,gamma))
        ss.append(ss[-1] + step_size*ds(ss[-1],ins[-2],pop,r0,gamma))
    return np.vstack([ins, ss])


def hospital_stress_delta(delta,i0,pop=POP,r0=R0,thresh=2.5e-3,step_size=1e-5):
    re0 = i0*(1-1/r0**5)*(1-1/r0) - i0
    s1 = simulate(50,i0,pop-i0-re0,pop,r0,step_size=step_size)
    s2 = simulate(50,i0+delta,pop-i0-delta-re0,pop,r0,step_size=step_size)
    inds1, inds2 = s1[0]>=pop*thresh,s2[0]>=pop*thresh
    
    return  (s2[0][inds2]-pop*thresh).sum()*step_size - (s1[0][inds1]-pop*thresh).sum()*step_size

def infections_delta(delta,i0,pop=POP,r0=R0,thresh=2.5e-3,step_size=1e-5):
    re0 = i0*(1-1/r0**5)*(1-1/r0) - i0
    s1 = simulate(50,i0,pop-i0-re0,pop,r0,step_size=step_size)
    s2 = simulate(50,i0+delta,pop-i0-delta-re0,pop,r0,step_size=step_size)
    
    return  min(s1[1]) - min(s2[1])

In [715]:
rates = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1]

# Probably not reliable when 10% of pop is infected

for rate in rates:
    print(f'Hospital stress 10 infections against {rate} background rate: {hospital_stress_delta(10,POP*rate):.2f}\n\
            Additional infections same parameters: {infections_delta(10,POP*rate):.2f}')

Hospital stress 10 infections against 1e-06 background rate: 4.90
            Additional infections same parameters: 0.49
Hospital stress 10 infections against 1e-05 background rate: 4.90
            Additional infections same parameters: 0.49
Hospital stress 10 infections against 0.0001 background rate: 4.90
            Additional infections same parameters: 0.49
Hospital stress 10 infections against 0.001 background rate: 4.90
            Additional infections same parameters: 0.49
Hospital stress 10 infections against 0.01 background rate: 1.63
            Additional infections same parameters: 0.49
Hospital stress 10 infections against 0.1 background rate: 0.56
            Additional infections same parameters: 0.42


for 0.01 and 0.1 background rate, we're seeing the effect of people entering hospital when infections are already past stress threshold